# Functioneer Examples

This notebook provides a comprehensive set of examples for the `functioneer` library, demonstrating core and advanced features. For a quick introduction and key examples, see the [README.md](README.md). We use the Rosenbrock function due to its simplicity, many inputs, and its popular use as an optimization benchmark.

Currently this notebook is the primary form of documentation. By the end you will have witnessed the computational power of this fully armed and fully operational library.

## Table of Contents
1. [Forks and Function Evaluation (The Basics)](#example-1)
2. [Optimization](#example-2)
3. [Multi-parameter Forks](#example-3)
4. [Defining Parameters](#example-4)
5. [Dictionary Output in Evaluate](#example-5)
6. [Evaluatee Options](#example-6)
7. [Conditionally Skip Analysis Steps](#example-7)
8. [Optimizer Options](#example-8)
9. [Custom Optimizer](#example-9)
10. [Runtime and Datetime Fields](#example-10)
11. [Invalid Define with Dictionary and Value](#example-11)

In [25]:
# Setup: Import libraries and define example functions
import functioneer as fn
import time

# Rosenbrock function (known minimum: value=0 @ x=1, y=1, a=1, b=100)
def rosenbrock(x, y, a, b):
    return (a - x)**2 + b * (y - x**2)**2


### <span id='example-1'>Example 1: Forks and Function Evaluation (The Basics)</span>

**Goal**: Test `rosenbrock` function with multiple values for parameters `x` and `y`.

Note: forks for `x` and `y` create a 'grid' of values\
Note: Parameter IDs MUST match your function's args, function evals inside functioneer are fully keyword arg based.

In [26]:
analysis = fn.AnalysisModule() # Create new analysis
analysis.add.define({'a': 1, 'b': 100}) # define a and b
analysis.add.fork('x', (0, 1, 2)) # Fork analysis, create branches for x=0, x=1, x=2
analysis.add.fork('y', (1, 10))
analysis.add.evaluate(func=rosenbrock) #
results = analysis.run()
print('Example 1 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosenbrock']])

Example 1 Output:
   a    b  x   y  rosenbrock
0  1  100  0   1         101
1  1  100  0  10       10001
2  1  100  1   1           0
3  1  100  1  10        8100
4  1  100  2   1         901
5  1  100  2  10        3601


### <span id='example-2'>Example 2: Optimization</span>

**Goal**: Optimize `x` and `y` to find the minimum `rosenbrock` value for various `a` and `b` values.

Note: values for `x` and `y` before optimization are used as initial guesses

In [27]:
analysis = fn.AnalysisModule({'x': 0, 'y': 0})
analysis.add.fork('a', (1, 2))
analysis.add.fork('b', (0, 100, 200))
analysis.add.optimize(func=rosenbrock, opt_param_ids=('x', 'y'))
results = analysis.run()
print('\nExample 2 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosenbrock']])


Example 2 Output:
   a    b         x         y    rosenbrock
0  1    0  1.000000  0.000000  4.930381e-32
1  1  100  0.999763  0.999523  5.772481e-08
2  1  200  0.999939  0.999873  8.146869e-09
3  2    0  2.000000  0.000000  0.000000e+00
4  2  100  1.999731  3.998866  4.067518e-07
5  2  200  1.999554  3.998225  2.136755e-07


### <span id='example-3'>Example 3: Multi-parameter Forks</span>

**Goal**: Test specific combinations of `x` and `y`: `(x=0, y=0), (x=1, y=10), (x=2, y=20)`.

Note: If defining fork with *value sets*, value sets must have matching length\
Note: Alternatively, you can take full manual control and define each branch as a *parameter configuration*.

In [ ]:
analysis = fn.AnalysisModule({'a': 1, 'b': 100})
analysis.add.fork({'x': (0, 1, 2), 'y': (0, 10, 20)}) # Fork multiple parameters with value sets
# analysis.add.fork(({'x': 0, 'y': 0}, {'x': 1, 'y': 10}, {'x': 2, 'y': 20})) # Fork with parameter configurations (alternate method)
analysis.add.evaluate(func=rosenbrock)
results = analysis.run()
print('\nExample 3 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosenbrock']])


Example 3 Output:
   a    b  x   y  rosenbrock
0  1  100  0   0           1
1  1  100  1  10        8100
2  1  100  2  20       25601


### <span id='example-4'>Example 4: Defining Parameters</span>

**Goal**: Demonstrate various ways to define parameters.

Note: a parameter is overwritten by any subsequent step setting that parameter

In [29]:
analysis = fn.AnalysisModule({'a': 0, 'b': 0, 'x': 0}) # define initial parameter values in AnalysisModule declaration
analysis.add.define('a', 1) # single parameter definition
analysis.add.define({'b': 100, 'y': 1}) # multi parameter definition
analysis.add.evaluate(func=rosenbrock)
results = analysis.run()
print('\nExample 4 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosenbrock']])


Example 4 Output:
   a    b  x  y  rosenbrock
0  1  100  0  1         101


### <span id='example-5'>Example 5: Dictionary Output in Evaluate</span>

Useful for functions that return multiple values

**Goal**: Evaluate a function returning a dictionary and unpack results.

Note: `unpack_result=True` stores dictionary keys as separate parameters

In [30]:
def rosenbrock_dict(x, y, a, b):
    return {'rosen': rosenbrock(x, y, a, b), 'sum': x + y}

analysis = fn.AnalysisModule({'a': 1, 'b': 100, 'x': 1, 'y': 1})
analysis.add.evaluate(func=rosenbrock_dict, unpack_result=True)
results = analysis.run()
print('\nExample 5 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosen', 'sum']])


Example 5 Output:
   a    b  x  y  rosen  sum
0  1  100  1  1      0    2


### <span id='example-6'>Example 6: Evaluate Options</span>

**Goal**: Demonstrate different combinations of `assign_to` and `unpack_result` in `evaluate`.

Note: 
- omitting `assign_to` uses the function name as the parameter ID
- `unpack_result=True` requires a dictionary output
- `unpack_result=True` with `assign_to=None` will unpack ALL dict keys 
- `unpack_result=True` with `assign_to=list[str]` will filter dict keys

In [31]:
def rosenbrock_dict(x, y, a, b):
    return {'rosen': rosenbrock(x, y, a, b), 'sum': x + y, 'prod': x * y}

analysis = fn.AnalysisModule({'a': 1, 'b': 100})
analysis.add.fork('x', (0, 1))
analysis.add.define('y', 1)
analysis.add.evaluate(func=rosenbrock)  # No assign_to
analysis.add.evaluate(func=rosenbrock, assign_to='brock_purdy')  # With assign_to
analysis.add.evaluate(func=rosenbrock_dict, assign_to=['rosen', 'prod'], unpack_result=True)  # Unpack and filter with assign_to
results = analysis.run()
print('\nExample 6 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosenbrock', 'brock_purdy', 'rosen', 'prod']])


Example 6 Output:
   a    b  x  y  rosenbrock  brock_purdy  rosen  prod
0  1  100  0  1         101          101    101     0
1  1  100  1  1           0            0      0     1


### <span id='example-7'>Example 7: Conditionally Skip Analysis Steps</span>

Any analysis step can be conditionally skipped based on parameter values at runtime.

**Goal**: Skip an expensive function evaluation if `y` is below 0.5 after optimization.

Note: condition functions must return a boolean

In [32]:
def expensive_func(x, y):
    time.sleep(0.2)  # delay to simulate expensive func
    return x + y

analysis = fn.AnalysisModule({'x': 0, 'y': 0, 'a': 1, 'b': 100})
analysis.add.fork('a', (1, 2))
analysis.add.fork('b', (0, 100, 200))
analysis.add.optimize(func=rosenbrock, opt_param_ids=('x', 'y'))
analysis.add.evaluate(func=expensive_func, condition=lambda y: y > 0.5)
results = analysis.run()
print('\nExample 7 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosenbrock', 'expensive_func', 'runtime']])


Example 7 Output:
   a    b         x         y    rosenbrock  expensive_func   runtime
0  1    0  1.000000  0.000000  4.930381e-32             NaN  0.002002
1  1  100  0.999763  0.999523  5.772481e-08        1.999286  0.218684
2  1  200  0.999939  0.999873  8.146869e-09        1.999811  0.207836
3  2    0  2.000000  0.000000  0.000000e+00             NaN  0.001903
4  2  100  1.999731  3.998866  4.067518e-07        5.998596  0.210156
5  2  200  1.999554  3.998225  2.136755e-07        5.997779  0.213020


### <span id='example-8'>Example 8: Optimizer Options</span>

**Goal**: Maximize `rosenbrock_neg` with Nelder-Mead and custom bounds/tolerance.

Note: `direction='max'` finds function maximum (default is `'min'`)

In [33]:
# Negative Rosenbrock for maximization
def rosenbrock_neg(x, y, a, b):
    return -rosenbrock(x, y, a, b)

analysis = fn.AnalysisModule({'a': 1, 'b': 100, 'x': 5, 'y': 4})
analysis.add.optimize(
    func=rosenbrock_neg,
    opt_param_ids=('x', 'y'),
    assign_to='rosen_neg', # set custom param name with assign_to
    direction='max',
    optimizer='Nelder-Mead',
    bounds={'x': (-100, 100), 'y': (-50, 50)},
    tol=1e-2
)
results = analysis.run()
print('\nExample 8 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosen_neg']])


Example 8 Output:
   a    b         x         y  rosen_neg
0  1  100  1.001503  1.002813  -0.000006


### <span id='example-9'>Example 9: Custom Optimizer</span>

**Goal**: Use a custom optimizer for `rosenbrock`.

Note: custom optimizer must return a dictionary matching the `scipy.optimze.OptimizeResult` format: (`x`, `fun`, `success` keys)

In [34]:
from scipy.optimize import minimize, OptimizeResult
def custom_optimizer(func: callable, x0: list, **kwargs) -> OptimizeResult:
    return minimize(func, x0, method='BFGS', options={'gtol': 1e-6})

analysis = fn.AnalysisModule({'a': 1, 'b': 100, 'x': 5, 'y': 4})
analysis.add.optimize(func=rosenbrock, opt_param_ids=('x', 'y'), optimizer=custom_optimizer)
results = analysis.run()
print('\nExample 9 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosenbrock']])


Example 9 Output:
   a    b         x         y    rosenbrock
0  1  100  0.999996  0.999991  2.003596e-11


### <span id='example-10'>Example 10: Runtime and Datetime Fields</span>

Functioneer automatically adds fields: `runtime` tracks cumulative step execution time; `datetime` marks leaf completion

**Goal**: Show `runtime` and `datetime` fields in results.

Note: The `runtime` and `datetime` fields are reserved parameter ids

In [35]:
analysis = fn.AnalysisModule({'a': 1, 'b': 100})
analysis.add.fork('x', (0, 1))
analysis.add.evaluate(func=lambda x, a, b: time.sleep(0.1) or x, assign_to='result')
results = analysis.run()
print('\nExample 10 Output:')
print(results['df'][['a', 'b', 'x', 'result', 'runtime', 'datetime']])


Example 10 Output:
   a    b  x  result   runtime                   datetime
0  1  100  0       0  0.101053 2025-06-30 00:41:14.027957
1  1  100  1       1  0.100841 2025-06-30 00:41:14.128798


### <span id='example-11'>Example 11: Invalid Define with Dictionary and Value</span>

**Goal**: Showcase error handling for invalid `define` input.

Note: providing `value` with a dictionary input raises a `ValueError`

In [36]:
analysis = fn.AnalysisModule({'a': 1})
try:
    analysis.add.define({'a': 1, 'b': 100}, value=2)
except ValueError as e:
    print(f'Example 11 Output: Caught expected error: {str(e)}')

Example 11 Output: Caught expected error: When defining multiple parameters with a dictionary, the 'value' argument is ignored. Use either define(param_id: str, value: Any) or define(params: Dict[str, Any]).
